# Paquetes

In [ ]:
# pip install geopandas

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 14.7MB 292kB/s 
     |████████████████████████████████| 10.9MB 44.5MB/s 


In [ ]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.ensemble import RandomForestRegressor

import sklearn
import matplotlib.pyplot as plt

# Para Mapas
import plotly.express as px
import branca
import geopandas
import folium

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


para cargar desde drive


In [ ]:
Data_Base = pd.read_csv("Data_Base_1419.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Modelo

In [ ]:
Data_Base2 = Data_Base.copy()
Data_Base1 = Data_Base2[Data_Base2.Ano < 2019]
Data_Base1 = Data_Base1[~Data_Base1.isin([np.nan, np.inf, -np.inf]).any(1)]
Data_Base2.columns

Index(['COLE_COD_MCPIO_UBICACION', 'Ano', 'FAMI_TIENEINTERNET',
       'FAMI_TIENECOMPUTADOR', 'ESTU_TIENEETNIA', 'COLE_NATURALEZA',
       'PUNT_GLOBAL', 'MUNICIPIO', 'DEPARTAMENTO', 'PoblacionTotal',
       'ConexMilHab', 'NoAccesosFijos', 'Indice_Rural', 'Riesgo'],
      dtype='object')

In [ ]:
UmbralDefault = math.ceil(Data_Base[Data_Base.Ano == 2019]['PUNT_GLOBAL'].mean() - 1 * Data_Base[Data_Base.Ano == 2019]['PUNT_GLOBAL'].std())
QuanUmbral = stats.percentileofscore(Data_Base[Data_Base.Ano == 2019]['PUNT_GLOBAL'], UmbralDefault) / 100


213.09025825121864

In [ ]:
Data_Base['Riesgo'] = 2
for i in [2014, 2015, 2016, 2017, 2018, 2019]:
    risk = Data_Base[Data_Base.Ano == i]['PUNT_GLOBAL'].quantile(QuanUmbral)
    Data_Base['Riesgo'] = np.where(i == Data_Base.Ano, np.where(Data_Base['PUNT_GLOBAL'] < risk, 1, 0), Data_Base.Riesgo)

Anno = 2019

fig = px.histogram(
    Data_Base[Data_Base.Ano == Anno],
    x="PUNT_GLOBAL",
    color="Riesgo",
    color_discrete_sequence=['#6495ED', '#FFA500'],
    category_orders={"Riesgo": [0, 1]},
    # labels={'Puntaje Global':'PUNT_GLOBAL'},
    # color_continuous_scale=['#FFA500', '#FFA500'],
    nbins=150
    )
fig.show()

In [ ]:
Data_Base1 = Data_Base2[Data_Base2.Ano < 2019]
Data_Base1 = Data_Base1[~Data_Base1.isin([np.nan, np.inf, -np.inf]).any(1)]

# Modelo LOGIT
Data_Base1['Intercepto'] = 1
variables = [
    'Intercepto',
    'FAMI_TIENEINTERNET', 
    'FAMI_TIENECOMPUTADOR',
    'ESTU_TIENEETNIA',
    'COLE_NATURALEZA',
    'ConexMilHab',
    'PoblacionTotal',
    'Indice_Rural'
    ]

logit1 = sm.Logit(Data_Base1['Riesgo'], Data_Base1[variables])
logit1_res = logit1.fit()

var_tmp = variables
while 0 != sum(logit1_res.pvalues > 0.05):
    T = len(logit1_res.pvalues)
    #print(logit1_res.pvalues.sort_values(ascending=True).reset_index()[-1:]['index'])
    var_tmp = logit1_res.pvalues.sort_values(
        ascending=True).reset_index()[:(T-1)]['index']
    logit1 = sm.Logit(Data_Base1['Riesgo'], Data_Base1[var_tmp])
    logit1_res = logit1.fit()

variables1 = var_tmp
pscore = logit1_res.predict(Data_Base1[variables1])
Data_Base1['pscore'] = pscore

# REGRESSION TREE
rf_modelD = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)
rf_modelD.fit(Data_Base1[variables1], Data_Base1['Riesgo'])
pscore_forestd = rf_modelD.predict(Data_Base1[variables1])
Data_Base1['pscore_forestd'] = pscore_forestd

# RANDOM FOREST
rf_model = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)
rf_model.fit(Data_Base1[variables1], Data_Base1['Riesgo'])
pscore_forest = rf_model.predict(Data_Base1[variables1])
Data_Base1['pscore_forest'] = pscore_forest

# Estimación
Test = Data_Base[Data_Base.Ano == 2019]
Test = Test[~Test.isin([np.nan, np.inf, -np.inf]).any(1)]

Test['Intercepto'] = 1
Test['riesgo_forest'] = rf_model.predict(Test[variables1])
Test['riesgo_forest'] = np.where(Test.riesgo_forest < 0.5, 0, 1)
Test['riesgo_regression'] = rf_modelD.predict(Test[variables1])
# Test['riesgo_regression'] = np.where(Test.riesgo_regression < 0.5, 0, 1)
Test['riesgo_logit'] = logit1_res.predict(Test[variables1])
Test['riesgo_logit'] = np.where(Test.riesgo_logit < 0.5, 0, 1)

Test['Riesgo_total'] = Test.riesgo_forest+Test.riesgo_logit+Test.riesgo_regression

Optimization terminated successfully.
         Current function value: 0.292766
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.292786
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.292953
         Iterations 8
